In [1]:
import fiftyone as fo
from client.yolov7_client.yolov7_triton_client import YoloV7_Triton_Inference_Client
from client.detectron2_client.detectron2_triton_client import Detectron2_Triton_Client

In [ ]:
# start the triton inference server, either locally or on a server

# docker run --gpus all --rm --ipc=host --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864 \
# -p8000:8000 -p8001:8001 -p8002:8002 --mount type=bind,source="path/to/triton/models",destination=/models \
# nvcr.io/nvidia/tritonserver:22.06-py3 tritonserver --model-repository=/models --strict-model-config=false \
# --log-verbose 1

In [19]:
# initialize the client
yolov7_client = YoloV7_Triton_Inference_Client()
detectron2_client = Detectron2_Triton_Client(width=1344, height=1344)

In [4]:
# start fiftyone

session = fo.launch_app(auto=False)
session.open_tab()

Connected to FiftyOne on port 5151 at localhost.
If you are not connecting to a remote session, you may need to start a new session and specify a port
Session launched. Run `session.show()` to open the App in a cell output.


<IPython.core.display.Javascript object>

In [22]:
# load dataset
dataset_name = "test_run1"
dataset = fo.load_dataset(dataset_name)
view = dataset.select(session.selected)

# delete selected samples
dataset.delete_samples(view)

In [21]:
# run the client 

# yolov7_client.infer_dummy()
yolov7_client.infer_image(input_='data/raw/images', fo_dataset="test_run1")
# yolov7_client.infer_video(input_='data/raw/video', output_='data/inference/video')

# detectron2_client.infer_dummy()
# detectron2_client.infer_image(input_='data/raw/images/test')
# detectron2_client.infer_video(input_='data/raw/video', output_='data/inference/video')

Running in 'image' mode
Done
Creating buffer from image file...
Invoking inference...


Received result buffer "num_dets" of size (1, 1)
Naive buffer sum: 1
Received result buffer "det_boxes" of size (1, 100, 4)
Naive buffer sum: 1189.5
Received result buffer "det_scores" of size (1, 100)
Naive buffer sum: 0.9648606777191162
Received result buffer "det_classes" of size (1, 100)
Naive buffer sum: 46
Detected objects: 2
Creating buffer from image file...
Invoking inference...
Received result buffer "num_dets" of size (1, 1)
Naive buffer sum: 3
Received result buffer "det_boxes" of size (1, 100, 4)
Naive buffer sum: 3692.75
Received result buffer "det_scores" of size (1, 100)
Naive buffer sum: 2.823911428451538
Received result buffer "det_classes" of size (1, 100)
Naive buffer sum: 24
Detected objects: 2
Creating buffer from image file...
Invoking inference...
Received result buffer "num_dets" of size (1, 1)
Naive buffer sum: 14
Received result buffer "det_boxes" of size (1, 100, 4)
Naive buffer sum: 20810.3125
Received result buffer "det_scores" of size (1, 100)
Naive buffe